In [2]:
import ast
import pickle
import os
import pandas as pd

#### Luigi migration. Task yet to be created

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join("../../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from jupyter_notebook import load_parameters 


pars = load_parameters()

so_dump_processed_file = pars.get('input')
output_file = pars.get('output')

input_col = pars.get('input_col')

api_doc_file = pars.get('api_doc_file')

debug = pars.get('debug')

### Utility to extract method name and line number

In [4]:
from collections import deque

class FunctionCallVisitor(ast.NodeVisitor):
    def __init__(self):
        self._name = deque()
        self._pos = -1 
    
    @property
    def name(self):
        return '.'.join(self._name)
    
    @property
    def lineno(self):
        return self._pos
    
    @name.deleter
    def name(self):
        self._name.clear()
    
    def visit_Name(self, node):
        self._pos = node.lineno # line number
        self._name.appendleft(node.id)
    
    def visit_Attribute(self, node):
        try:
            self._pos = node.lineno # line number
            self._name.appendleft(node.attr)
            self._name.appendleft(node.value.id)
        except AttributeError:
            self.generic_visit(node)
            
def get_func_calls(tree):
    func_calls = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Call):
            callvisitor = FunctionCallVisitor()
            callvisitor.visit(node.func)
            func_calls.append((callvisitor.name, callvisitor.lineno))
    return func_calls

#### Load APIDoc and Dataset

In [6]:
#api_doc_file = '../../data-import/build_api_doc_base/api_doc.csv'
#so_dump_processed_file = '../../../data/stack-overflow/pandas-preprocessedcode-dataset-part3'
#input_col = 'PreprocessedCode3'
#output_file = '../../../data/stack-overflow/pandas-solutioncode-h1'

api_df = pd.read_csv(api_doc_file, encoding='ISO-8859-1', error_bad_lines=False)

processed_stackoverflow_df = pd.read_pickle(so_dump_processed_file)

def buildAPIDictionary(api_df):
    api_dict = dict()
    try:
        for index, row in api_df.iterrows():
            methodContext = row['Description']
            tokens = row['FullyQualifiedName'].split('.')
        
            for token in tokens:
                methodContext = str(methodContext)+' '+token
            api_dict[row['MethodName']] = methodContext
    except Exception as e:
        print('Error in method buildAPIDictionary',e)
    return api_dict

    
api_dict = buildAPIDictionary(api_df)

In [7]:
def lookUpAPIDocForContext(method_name):
    try:
        if method_name in api_dict.keys():
            return True
        else:
            return False
    except Exception as e:
        print('Error in method lookUpAPIDocForContext', e)

### H1 Method

Look for API calls in Code Fragment:
After analyzing the stackoverflow answers, it has been
identified that, usually solutions are provided in terms of API
method calls.

In [19]:
def H1(df):
    
    # Do not process questions
    if df.PostTypeId == 1:
        df['Solution'] = 'NA'
    
    else:
        # Parse code and inspect the function
        code_snippet = df[input_col]
        tree = ast.parse(code_snippet)        
        func_calls = get_func_calls(tree)

        solution_lines = set()
        for function, lineno in func_calls:   
            # print(function)
            tokens = function.split('.')
            method_name = tokens[len(tokens)-1]
            if lookUpAPIDocForContext(method_name):
                # Use a set to not add the same line twice
                solution_lines.add(lineno - 1)
                
        solution = []
        snippet_per_line = code_snippet.split(os.linesep)
        solution_lines = sorted(solution_lines)
        for i in solution_lines:
            solution.append(snippet_per_line[i])
                
                
        #print(solution)
        df['Solution'] = solution
        #df['Solution'] = os.linesep.join(solution)
    return df


solution_df = processed_stackoverflow_df.apply(H1, axis=1)


for index, row in solution_df.iterrows():
    if row['Solution'] != 'NA':
        print(row['Id'], row['Solution'])


7779260 []
7837947 ['    pct_change.append(...)']
7849789 ['for date, row in df.T.iteritems():', '    n = len(dates)']
8916746 ['df[(df.values > 1.5).any(1)]']
8992714 []
8997908 []
9555766 []
9557319 []
9558852 []
9577305 []
9620832 ['data_2010 = pandas.read_csv("/path/to/2010.csv")', 'data_2010.groupby("category").agg([len, sum])']
9623878 ["df.pivot_table(rows='category', aggfunc=[len, np.sum])", "df.pivot_table(rows='category', aggfunc=[len, np.sum])"]
9652858 ["DataFrame.from_csv('c:/~/trainSetRel3.txt', sep='\\t')", "DataFrame.from_csv('c:/~/trainSetRel3.txt', sep='\\t', header=0)"]
9656288 []
9677723 []
9762084 []
9772031 []
9794891 ['df2.combine_first(df1)']
10114652 ['merge(left, right)', "merge(left, right, on='ST_NAME', sort=False)"]
10182172 []
10202789 ["df = pandas.DataFrame(np.random.randn(5,3),columns=['A','B','C'])", "df['A'].argmax()", "df['B'].argmax()", "df['C'].argmax()", "dfrm['A'].idxmax()", "dfrm.ix[dfrm['A'].idxmax()]"]
10213167 ["df = pandas.DataFrame(np.rando

17211698 []
17216674 []
17241104 []
17242374 ["df = pd.DataFrame(index=['a', 'b'])"]
17243346 ['g.index = g.index.swaplevel(1, 2)', 'g.index = g.index.swaplevel(1, 2)', "g = df.groupby(['Manufacturer', 'Product Launch Date', 'Product Name']).sum()"]
17244095 ['df.index.get_loc(ds)']
17287046 ['df[df > df.quantile(0.8)].dropna()', 'list(df[df > df.quantile(0.8)].dropna().index)']
17298454 ["medals = df.pivot_table('no of medals', ['Year', 'Country'], 'medal')", "medals.reindex_axis(['Gold', 'Silver', 'Bronze'], axis=1)"]
17302673 []
17315875 []
17322585 ["df = DataFrame(randn(20,4),columns=list('ABCD'))", "df[(df['A']>0) & (df['B']>0) & (df['C']>0)].count()", "len(df[(df['A']>0) & (df['B']>0) & (df['C']>0)])"]
17347945 []
17383140 []
17383325 ['df = DataFrame(dict(A = True, B = False),index=range(3))', 'df.astype(int)', 'df.astype(int).dtypes']
17426500 ['Series(df.Letter.values,index=df.Position).to_dict()', "df = DataFrame(randint(0,10,10000).reshape(5000,2),columns=list('AB'))"]
1743

22992568 ['df[df[\'stridx\'].str.contains("Hello|Britain")]']
23005564 []
23088780 ['df1 = pd.read_fwf(io.BytesIO(texts[0]), widths=[5,7,25,17,20], parse_dates=[4])', 'df2 = pd.read_fwf(io.BytesIO(texts[1]), widths=[5,7,25,17,20], parse_dates=[4])', "    return x[0] if x[0] == x[1] else '{} | {}'.format(*x)", 'my_panel = pd.Panel(dict(df1=df1,df2=df2))', '    elif pd.isnull(x[0]) and pd.isnull(x[1]):', '    elif pd.isnull(x[0]) and ~pd.isnull(x[1]):', '    elif ~pd.isnull(x[0]) and pd.isnull(x[1]):', 'HTML(my_panel.apply(report_diff, axis=0).to_html(escape=False))']
23104436 ["df.to_sql('table_name', engine)"]
23112008 []
23143081 ["df['dA'] = df['A'] - df['A'].shift(-1)"]
23143110 ['df = pd.DataFrame({"A": [9, 4, 2, 1], "B": [12, 7, 5, 4]})', 'df["dA"] = df["A"].diff(-1)']
23146038 []
23151722 ['row_iterator = df.iterrows()']
23178185 []
23198160 ["df['beyer_shifted'] = df.groupby(level=0)['beyer'].shift(1); df"]
23200666 ["df=pd.DataFrame({'Data':np.random.normal(size=200)})  #exampl

31037360 ['df["weight"].mean()']
31061820 []
31075478 ["df['si_name'] = R.index.get_level_values('si_name') "]
31076657 ["frame[frame.filename.str.match('*.'+MetaData+'.*') & frame.file_path.str.match('C:\\test\\test.txt')]"]
31097813 ["series = pd.Series([1,2], index=['a','b'])", 'df = pd.DataFrame([series])', 'list_of_series = [pd.Series([1,2],index=cols), pd.Series([3,4],index=cols)]', 'df = pd.DataFrame(list_of_series, columns=cols)', "list_of_series = [pd.Series([1,2],index=['a','b']), pd.Series([3,4],index=['a','c'])]", 'df = pd.concat(list_of_series, axis=1).transpose()']
31105951 []
31173785 ["df = pd.DataFrame({'Type':list('ABBC'), 'Set':list('ZZXY')})"]
31206596 ["df = pd.DataFrame({'x': [1,2,3,4,5], "]
31247247 ["df.to_csv(r'c:\\data\\pandas.txt', header=None, index=None, sep=' ', mode='a')"]
31247279 []
31257931 []
31296878 ["table.query('column_name == some_value | column_name2 == some_value2')", 'df = pd.DataFrame(d)', "df.query('foo == 222 | bar == 444')"]
31297540 []
31

In [20]:
debug = True

solution_df.to_pickle(output_file)

if debug:
    solution_df.to_csv(output_file + ".csv", encoding='ISO-8859-1', sep=",", doublequote=True, index=False)
